## Тема “Создание признакового пространства”

### Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

### Задание 1.
1.1 Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)       
1.2 С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.



1.1

In [1]:
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
combine_df = pd.read_pickle('tweet_preprocessed.pkl')
combine_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[when, father, is, dysfunct, and, is, so, self...","[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, for, lyft, credit, can, not, use, caus...","[thanks, for, lyft, credit, can, not, use, cau..."


In [4]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head(2)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...


In [5]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head(2)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...


In [6]:
# !pip install -U spacy
#!python -m spacy info

import spacy
from spacy import displacy
import en_core_web_md

In [7]:
spacy.cli.download('en_core_web_md')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [8]:
train_len = train_df.shape[0]
test_len = test_df.shape[0]

In [9]:
nlp = en_core_web_md.load()
train_text = combine_df.clean_tweet[:train_len].apply(lambda x: nlp(x)).tolist()
test_text = combine_df.clean_tweet[train_len:].apply(lambda x: nlp(x)).tolist()
combine_text = train_text + test_text

# len(text_combine)
# article = nlp(text_combine)
# displacy.render(article, jupyter=True, style='ent')

In [ ]:
displacy.render(combine_text, jupyter=True, style='ent')

In [11]:
len(combine_text) == combine_df.shape[0]

True

In [12]:
word_NER = []
for doc in combine_text:
    for ent in doc.ents:
        word_NER.append((ent.text, ent.label_))

df_ner = pd.DataFrame(word_NER, columns=['word', 'word_NER'])
df_ner.head(10)

,word,word_NER
0,pdx,GPE
1,getthanked,PERSON
2,bihday,DATE
3,yoyour,GPE
4,tomorrow,DATE
5,danny,PERSON
6,the next school year,DATE
7,the year,DATE
8,cavs champions cleveland,ORG
9,ireland,GPE


In [13]:
df_ner.shape[0]

43378

## ТОП-20 популярных NER

In [14]:
df_ner.word_NER.value_counts().head(20)

PERSON         11639
DATE           10584
ORG             8305
GPE             5862
TIME            2223
NORP            1711
CARDINAL        1062
ORDINAL          576
FAC              376
PRODUCT          305
LOC              283
EVENT            220
WORK_OF_ART      106
QUANTITY          54
LANGUAGE          36
MONEY             19
LAW               13
PERCENT            4
Name: word_NER, dtype: int64

1.2

In [63]:
word_NER = []
for doc in train_text:
    for ent in doc.ents:
        word_NER.append((ent.text, ent.label_))

df_train_ner = pd.DataFrame(word_NER, columns=['word', 'word_NER'])


In [64]:
word_NER = []
for doc in test_text:
    for ent in doc.ents:
        word_NER.append((ent.text, ent.label_))

df_test_ner = pd.DataFrame(word_NER, columns=['word', 'word_NER'])


In [82]:
df_combine_ner = pd.concat([df_train_ner, df_test_ner], ignore_index=True)

df_combine_ner.head(10)

,word,word_NER
0,pdx,GPE
1,getthanked,PERSON
2,bihday,DATE
3,yoyour,GPE
4,tomorrow,DATE
5,danny,PERSON
6,the next school year,DATE
7,the year,DATE
8,cavs champions cleveland,ORG
9,ireland,GPE


In [67]:
len(df_combine_ner)

43378

## Наиболее обсуждаемые персоны в train датасете

In [68]:
df_train_ner.loc[df_train_ner['word_NER'] == 'PERSON'].word.value_counts().head(20)

bing bong bing bong                                        75
obama                                                      45
suppo                                                      35
hillary                                                    35
melancholy melancholymusic                                 31
sta                                                        26
bjp                                                        23
feminismiscancer feminismisterrorism feminismmuktbharat    20
hu                                                         18
christina grimmie                                          18
tgif ff                                                    18
sea shepherd suppoers                                      17
yoyoyou                                                    17
gamedev indiedev indiegamedev                              17
heabroken                                                  16
lebron                                                     16
jo cox  

## Наиболее обсуждаемые персоны в test датасете

In [69]:
df_test_ner.loc[df_test_ner['word_NER'] == 'PERSON'].word.value_counts().head(20)

bing bong bing bong                                        32
suppo                                                      31
obama                                                      22
feminismiscancer feminismisterrorism feminismmuktbharat    20
sta                                                        19
hillary                                                    13
christina grimmie                                          12
bjp                                                        11
jo cox                                                     11
jesus                                                      10
lebron                                                     10
regrann                                                    10
donald trump                                                9
clinton                                                     9
melancholy melancholymusic                                  9
th bihday                                                   9
boricua 

## Наиболее обсуждаемые организации в train датасете

In [86]:
df_train_ner.loc[df_train_ner['word_NER'] == 'ORG'].word.value_counts().head(20)

sjw                 73
allahsoil           52
gop                 51
nba                 38
islam               34
cavs                30
social analytics    27
ht                  27
wso                 26
trump               26
facebook            25
cnn                 24
google              21
ios                 21
bihday              18
eu                  17
house               15
brexit              14
obama               14
bogota colombia     14
Name: word, dtype: int64

## Наиболее обсуждаемые организации в test датасете

In [81]:
df_test_ner.loc[df_test_ner['word_NER'] == 'ORG'].word.value_counts().head(20)

allahsoil    31
sjw          29
gop          25
trump        21
bihday       20
nba          20
ht           17
islam        16
facebook     11
obama        11
amazon        9
eu            9
cnn           9
cavs          9
google        8
fbi           8
ios           8
gbpusd        8
vsco          8
spo           8
Name: word, dtype: int64

### Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [102]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [108]:
# nltk.pos_tag(nltk.word_tokenize(document))

In [103]:
document = combine_df.clean_tweet.apply(lambda x: nlp(x)).to_string()
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

set()

### Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


NLTK без результата